In [28]:
import os
import numpy as np 
import pandas as pd
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import spacy
import google.generativeai as genai
#from chatlas import ChatGoogle, ChatOpenAI
from langchain_openai import ChatOpenAI as LangchainChatOpenAI 
import re 
from spacytextblob.spacytextblob import SpacyTextBlob
import matplotlib.pyplot as plt
import seaborn as sns
### Para gráficos
import plotly.express as px
import plotly.graph_objects as go
from scipy.special import softmax
from pysentimiento import create_analyzer
import google.generativeai as genai
from deep_translator import GoogleTranslator, single_detection

from dotenv import load_dotenv

In [29]:
load_dotenv()   

True

In [30]:
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

In [31]:
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')


In [32]:
import pandas as pd 
mainPath = 'E:/Users/1167486/Local/proyectos/divisas/Scraper_divisas'

test = pd.read_csv(mainPath + '/playstore_scraped_data_divisas.csv')


In [33]:
text = ' '.join(test['content'])

In [34]:

#model = genai.GenerativeModel('gemini-2.0-flash')

prompt = f'''
Analiza el siguiente texto:
"{text}"

Tu tarea es extraer:
1. Funcionalidades clave (tanto positivas como negativas). Estas deben ser frases concisas, por ejemplo: 'App falla en depósitos', 'Interfaz intuitiva y rápida'.
2. Diez (10) categorías relevantes y sencillas (máximo 4 palabras cada una) que resuman los temas principales o aspectos mencionados en el texto.

IMPORTANTE: Formatea TODA tu respuesta EXCLUSIVAMENTE como una ÚNICA cadena de texto que represente una lista de Python.
Esta lista debe contener strings. Cada string puede ser una funcionalidad o una categoría.
Ejemplo de formato de respuesta esperado: ['funcionalidad A', 'funcionalidad B', 'categoría X', 'categoría Y', ..., 'categoría Z']
No incluyas NINGÚN texto, explicación, ni markdown (como ```python ... ```) antes o después de esta lista. Solo la lista en formato de cadena.
'''
prompt = model.generate_content(prompt)

In [35]:
prompt.text.strip()

"['Interfaz intuitiva y rápida', 'Transferencias rápidas y fáciles', 'Buena tasa de cambio', 'Pagos internacionales sencillos', 'Tarjeta virtual útil para viajes', 'Problemas con depósitos', 'App falla en depósitos', 'Servicio al cliente deficiente', 'Bloqueo de cuentas sin explicación', 'Promociones engañosas', 'Retrasos en las transferencias', 'Comisiones altas', 'Problemas con retiros', 'Falta de soporte', 'Seguridad deficiente', 'Verificación de cuentas compleja', 'Funcionalidades limitadas por país', 'Problemas con la tarjeta física', 'Dificultades para cerrar cuentas', 'Funcionalidad con PIX', 'Envío y recepción de dinero', 'Compra y venta de divisas', 'Ahorro en dólares', 'Pagos con QR', 'Inversión en criptomonedas', 'Cuentas remuneradas', 'Transferencias internacionales', 'Atención al cliente', 'Seguridad de la app', 'Promociones y recompensas', 'Funcionalidad de la tarjeta', 'Experiencia del usuario', 'Disponibilidad de divisas', 'Tarifas y comisiones', 'Verificación de identi

In [36]:
nlp_model = spacy.load('es_core_news_md')
nlp_model.add_pipe('spacytextblob')
sentiments_analizer = create_analyzer(task="sentiment", lang="es")
emotions_analizer = create_analyzer(task="emotion", lang="es")


In [ ]:
def generate_sentiment_analysis(text, analizer=None):
    text = str(text)
   
    if nlp_model:
        doc = nlp_model(text)
        polarity_spacy = doc._.blob.polarity

        if polarity_spacy > 0.1:
            return 'Positivo'  
        elif polarity_spacy < -0.1:
             return 'Negativo' 
        else:
            sentiment_spacy_neutral = 'Neutral'
            print(f"SpacyTextBlob polarity ({polarity_spacy:.3f}) is neutral/weak. Consulting Pysentimiento for: '{text[:60]}...'")

            analyzer_pysent = analizer

            if analyzer_pysent:
                try:
                    result_pysentimiento = analyzer_pysent.predict(text)
                    pysent_sentiment_map = {"POS": "Positivo", "NEG": "Negativo", "NEU": "Neutral"}
                    sentiment_pysent = pysent_sentiment_map.get(result_pysentimiento.output, "Neutral")

                    if sentiment_pysent != "Neutral":
                        print(f"Pysentimiento valores: {sentiment_pysent} (Probas: {result_pysentimiento.probas})")
                        return sentiment_pysent
                    else:
                        print(f"Pysentimiento también dice que es neutral: {result_pysentimiento.probas}")
                        return sentiment_spacy_neutral
                except Exception as e:
                    print(f"Error en pysentimiento: {e}. Tomando el valor de Spacy ({sentiment_spacy_neutral}).")
                    return sentiment_spacy_neutral
            else: 
                print("Pysentimiento no disponible. Se queda valor neutro")
                return sentiment_spacy_neutral
    else:
        print("Spacy falló, usar Pysentimiento.")
        analyzer_pysent = analizer
 
        if analyzer_pysent:
            try:
                result = analyzer_pysent.predict(text)
                pysent_sentiment_map = {"POS": "Positivo", "NEG": "Negativo", "NEU": "Neutral"}
                return pysent_sentiment_map.get(result.output, "Neutral")
            except Exception as e:
                print(f"Error en pysentimiento: {e}")
                return "Error: Análisis de sentimiento (Pysentimiento) fallido"
        else: # Both Spacy and Pysentimiento are unavailable
            return "Error: Modelos de sentimiento no disponibles"        


def generate_emotions_analysis(text, analizer=None):
    text = str(text)
    emotions_analyzer =analizer
    if emotions_analyzer is None:
        return "Error interno modelo de emociones"
    emotion_map_es = {
        "joy": "Alegría", 
        "sadness": "Tristeza", 
        "anger": "Enojo", 
        "fear": "Miedo", 
        "surprise": "Sorpresa", 
        "disgust": "Asco", 
        "neutral": "Neutral"
    }    
    try:
        result = emotions_analyzer.predict(text)
        primary_emotion_en = result.output
        if primary_emotion_en=="others":
            primary_emotion_en="neutral"
        return emotion_map_es.get(primary_emotion_en, "Desconocida")
    except Exception as e:
        print(f"Error en el análisis de emociones {e}")
        return "Error: Análisis de emociones fallido"

In [17]:
from transformers import pipeline
from tqdm.auto import tqdm
import time 
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

tqdm.pandas()

'''
def generate_zero_shot_classification(text, candidate_labels=None):
    text = str(text)
    if candidate_labels is None:
        return "Error interno modelo de clasificación"
    try:
        result = classifier(text, candidate_labels)
        return result['labels'][0]
    except Exception as e:
        #print(f"Error en el análisis de emociones {e}")
        print(f"No hay categoría que haga match para esta frase, se manda a default")
        return "No Aplica"
'''


def generate_zero_shot_classification(text, candidate_labels=None):
    # Ensure text is a string, handle potential None/NaN from DataFrame
    text = str(text) if pd.notna(text) else ""

    if not text: # Handle empty or NaN text after conversion
        return "Texto vacío o inválido" # Or some other indicator

    if candidate_labels is None:
        # This case indicates a programming error in how the function is called
        # For a DataFrame apply, candidate_labels should likely come from another column
        # or be a fixed list passed to the lambda/function.
        print("Error: candidate_labels is None") # Log this error
        return "Error interno modelo de clasificación"

    # Ensure candidate_labels is a list
    if not isinstance(candidate_labels, list):
         # Handle cases where candidate_labels might not be a list (e.g., from a DataFrame column)
         # You might need more specific handling depending on your data structure
         print(f"Warning: candidate_labels is not a list: {candidate_labels}. Attempting conversion.")
         try:
             # Try converting if it's a string representation of a list, or similar
             candidate_labels = eval(str(candidate_labels)) # Use eval cautiously or find a safer way
             if not isinstance(candidate_labels, list):
                  raise ValueError("Conversion did not result in a list")
         except Exception as e:
              print(f"Error processing candidate_labels: {e}")
              return "Error interno modelo de clasificación"


    try:
        # Add a small artificial delay to simulate work if needed for testing progress bar
        # time.sleep(0.01)
        result = classifier(text, candidate_labels)
        # The result structure is a dictionary, result['labels'] is a list of labels sorted by score
        return result['labels'][0] # Return the top label
    except Exception as e:
        # It's better to log the actual error for debugging
        # print(f"Error during classification: {e}")
        # The original message "No hay categoría que haga match..." is misleading
        # This catch block handles *any* error during the classification call,
        # not just a lack of matching categories (which the model handles internally).
        print(f"Error processing text because empty row '{text[:50]}...': {e}. Returning 'No Aplica'.")
        return "No Aplica" # Default value on error

In [99]:
generate_zero_shot_classification('estoy feliz', candidate_labels=['felicidad', 'tristeza', 'enojo'])

'felicidad'

In [12]:
def collapse_text(df, column_name):
    """
    Colapsa el texto en una sola celda.
    """
    if column_name in df.columns:
        return ' '.join(df[column_name].astype(str).tolist())
    else:
        raise ValueError(f"La columna '{column_name}' no existe en el DataFrame.")

In [13]:

def TranslateText(text, source, target):
    translatedText = GoogleTranslator(source = source, target = target).translate(text)
    return translatedText    


In [52]:
def translate_comment_safely(row, target_language='es', SUPPORTED_LANGUAGES= None):
    comment = row.get('comment') # Use .get() for safety, in case 'comment' key is missing
    detected_language = row.get('language_detected')

    # Condition 1: If comment is missing, empty, or language is not detected, return original (or None if comment was None)
    if not comment or not detected_language:
        return comment

    # Condition 2: If the language is already the target language, return original comment
    if detected_language == target_language:
        return comment

    # Condition 3: If the detected language is not supported for translation, return original comment
    if detected_language not in SUPPORTED_LANGUAGES:
        return comment

    # Condition 4: Attempt translation
    # You might want to add a try-except block here if TranslateText can raise errors
    try:
        translated_text = TranslateText(comment, source=detected_language, target=target_language)
    except Exception as e:
        # Optionally log the error e
        # print(f"Error translating comment from {detected_language}: {e}")
        return comment # Fallback to original comment on error

    # Condition 5: If translation result is None (e.g., API failed, or returned None for some reason), return original comment
    if translated_text is None:
        return comment
    
    return translated_text


In [14]:
print(generate_emotions_analysis(text="Estoy muy alegre y feliz por tener una mascota", analizer=emotions_analizer))
print(generate_sentiment_analysis(text="Estoy feliz", analizer=sentiments_analizer))

Alegría
SpacyTextBlob polarity (0.000) is neutral/weak. Consulting Pysentimiento for: 'Estoy feliz...'
Pysentimiento valores: Positivo (Probas: {'NEG': 0.12126108258962631, 'NEU': 0.4114501178264618, 'POS': 0.4672887623310089})
Positivo


In [15]:
'''
from huggingface_hub import snapshot_download

try:
    local_dir = snapshot_download(repo_id="imranali291/genderize")
    print(f"Modelo descargado localmente en: {local_dir}")
    # Luego intenta cargar desde esta ruta local
    model = AutoModelForSequenceClassification.from_pretrained(local_dir)
    tokenizer = AutoTokenizer.from_pretrained(local_dir)
    print("Modelo cargado desde la ruta local.")
except Exception as e:
    print(f"Error al descargar o cargar manualmente: {e}")
'''

'\nfrom huggingface_hub import snapshot_download\n\ntry:\n    local_dir = snapshot_download(repo_id="imranali291/genderize")\n    print(f"Modelo descargado localmente en: {local_dir}")\n    # Luego intenta cargar desde esta ruta local\n    model = AutoModelForSequenceClassification.from_pretrained(local_dir)\n    tokenizer = AutoTokenizer.from_pretrained(local_dir)\n    print("Modelo cargado desde la ruta local.")\nexcept Exception as e:\n    print(f"Error al descargar o cargar manualmente: {e}")\n'

In [16]:
import re 
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "imranali291/genderize"
modelGender = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example inference function
def predict_gender(name):
    inputs = tokenizer(name, return_tensors="pt", padding=True, truncation=True, max_length=32)
    outputs = modelGender(**inputs)
    predicted_label_id = outputs.logits.argmax(dim=-1).item()
    predicted_label = modelGender.config.id2label[predicted_label_id]
    predicted_label = str(predicted_label).replace("LABEL_", "")
    return 'Male' if predicted_label == '1' else 'Female'

In [120]:
import re
import pandas as pd
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Carga del modelo (asegúrate de que esté configurada globalmente o se pase)
model_name = "imranali291/genderize"
modelGender = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Puedes cargar una lista de nombres comunes para filtrado si lo deseas
# Por ejemplo, desde un archivo de texto o un conjunto de datos (Opcional)
# common_names = set(pd.read_csv('common_names.csv')['name'].str.lower())

def preprocess_twitter_username(username):
    if not isinstance(username, str) or not username.strip():
        return None

    # 1. Convertir a minúsculas para consistencia
    username_lower = username.lower()

    # 2. Eliminar números y caracteres especiales (excepto quizás underscores o guiones)
    # y dividir por ellos para obtener posibles "tokens" de nombre
    # Ejemplo: "juan_perez123" -> ["juan", "perez"]
    # Ejemplo: "maria.sol" -> ["maria", "sol"]
    # Ejemplo: "user-name" -> ["user", "name"]
    name_tokens = re.findall(r'[a-z]+', username_lower) # Extrae solo secuencias de letras

    # 3. Filtrar tokens no significativos (demasiado cortos, comunes, etc.)
    significant_tokens = []
    for token in name_tokens:
        if 2 < len(token) < 20 and token not in ['user', 'admin', 'official', 'bot', 'gaming', 'sports', 'fan', 'real', 'the', 'of', 'and', 'for', 'a', 'an']:
            significant_tokens.append(token)
            # Opcional: Si el primer token significativo es un nombre, úsalo y sal.
            # Esto prioriza el primer "nombre" que aparece.
            # if token in common_names: # Si usas un diccionario de nombres
            #     return token

    if significant_tokens:
        # Prioriza la primera palabra o la más larga que parezca un nombre
        # Aquí, simplemente tomamos la primera palabra significativa encontrada.
        return significant_tokens[0]
    
    return None # Si no se pudo extraer un nombre significativo

In [126]:
def predict_gender_from_username(username, confidence_threshold=0.7):
    """
    Predice el género a partir de un nombre de usuario de Twitter.

    Args:
        username (str): El nombre de usuario de Twitter.
        confidence_threshold (float): Umbral de confianza para clasificar como 'Male'/'Female'
                                      o 'Unknown'.

    Returns:
        dict: Un diccionario con 'gender' (Male, Female, Unknown) y 'confidence'.
    """
    processed_name = preprocess_twitter_username(username)

    if processed_name is None:
        return {'gender': 'Unknown', 'confidence': 0.0, 'reason': 'No significant name extracted'}

    inputs = tokenizer(processed_name, return_tensors="pt", padding=True, truncation=True, max_length=32)
    
    # Mover a GPU si está disponible (solo para rendimiento)
    # if torch.cuda.is_available():
    #     inputs = {k: v.to('cuda') for k, v in inputs.items()}
    #     modelGender.to('cuda')

    with torch.no_grad(): # Desactiva el cálculo de gradientes para inferencia
        outputs = modelGender(**inputs)
        probabilities = torch.softmax(outputs.logits, dim=-1)[0]
    
    predicted_label_id = probabilities.argmax().item()
    confidence = probabilities[predicted_label_id].item()

    predicted_label_str = str(modelGender.config.id2label[predicted_label_id]).replace("LABEL_", "")
    
    gender_prediction = 'Male' if predicted_label_str == '1' else 'Female'

    if confidence < confidence_threshold:
        return {'gender': 'Unknown', 'confidence': confidence, 'reason': 'Low confidence'}
    else:
        return {'gender': gender_prediction, 'confidence': confidence, 'reason': 'High confidence'}


def predict_gender_from_username(username, confidence_threshold=0.7):
    processed_name = preprocess_twitter_username(username)

    # Aseguramos que la función siempre devuelva un diccionario con todas las claves esperadas
    # incluso si el nombre no es válido o la confianza es baja.
    if processed_name is None:
        return {'gender': 'Unknown', 'confidence': 0.0, 'gender_liability': 'No significant name extracted'}

    inputs = tokenizer(processed_name, return_tensors="pt", padding=True, truncation=True, max_length=32)
    
    with torch.no_grad():
        outputs = modelGender(**inputs)
        probabilities = torch.softmax(outputs.logits, dim=-1)[0]
    
    predicted_label_id = probabilities.argmax().item()
    confidence = probabilities[predicted_label_id].item()

    # Manejar caso de que id2label no tenga la etiqueta esperada (poco probable si el modelo carga bien)
    predicted_label_str = 'Unknown_Label'
    if predicted_label_id in modelGender.config.id2label:
        predicted_label_str = str(modelGender.config.id2label[predicted_label_id]).replace("LABEL_", "")
    
    gender_prediction = 'Male' if predicted_label_str == '1' else 'Female'

    if confidence < confidence_threshold:
        return {'gender': 'Unknown', 'confidence': confidence, 'gender_liability': 'Low confidence'}
    else:
        return {'gender': gender_prediction, 'confidence': confidence, 'gender_liability': 'High confidence'}


In [123]:
predict_gender_from_username('@fertimjim')

{'gender': 'Male',
 'confidence': 0.8102343082427979,
 'reason': 'High confidence'}

In [ ]:
# Ejemplo de uso en un DataFrame de Twitter
# Asegúrate de que tu DataFrame socialMediaData tiene una columna 'username'

# Data de ejemplo para mostrar la funcionalidad
data_twitter = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8],
    'username': [
        "john_doe123",
        "maria_sol_mx",
        "el_profe_juan",
        "gaming_gurl",
        "official_ana",
        "supermanfan_99",
        "user_random_text",
        "notarealname"
    ],
    'comment_es': ["Comentario 1", "Comentario 2", "Comentario 3", "Comentario 4",
                   "Comentario 5", "Comentario 6", "Comentario 7", "Comentario 8"]
}
df_twitter_users = pd.DataFrame(data_twitter)

# Aplicar la función al DataFrame
df_twitter_users['gender_info'] = df_twitter_users['username'].apply(predict_gender_from_username)

# Expandir el diccionario 'gender_info' en columnas separadas
df_twitter_users = pd.concat([df_twitter_users.drop('gender_info', axis=1), 
                              df_twitter_users['gender_info'].apply(pd.Series)], axis=1)

print(df_twitter_users)

# Esto te dará columnas como 'gender', 'confidence', 'reason'

In [35]:
import pandas as pd
from transformers import pipeline

def detectar_idioma_df(df: pd.DataFrame, columna_texto: str, modelo_nombre: str = "papluca/xlm-roberta-base-language-detection") -> pd.DataFrame:
    """
    Detecta el idioma del texto en una columna específica de un DataFrame
    utilizando un modelo pre-entrenado de Hugging Face.

    Args:
        df (pd.DataFrame): El DataFrame de pandas.
        columna_texto (str): El nombre de la columna que contiene el texto a analizar.
        modelo_nombre (str): El nombre del modelo de Hugging Face a utilizar.
                             Por defecto: "papluca/xlm-roberta-base-language-detection".

    Returns:
        pd.DataFrame: El DataFrame original con una nueva columna 'idioma_detectado'
                      y 'confianza_idioma'.
    """
    # Inicializar el pipeline de clasificación de texto para detección de idioma
    # Aquí es importante que el modelo sea adecuado para la tarea de clasificación de idioma
    try:
        classifier = pipeline("text-classification", model=modelo_nombre)
    except Exception as e:
        print(f"Error al cargar el modelo '{modelo_nombre}': {e}")
        print("Asegúrate de que el modelo es adecuado para clasificación de texto/idioma.")
        return df

    # Función auxiliar para aplicar al DataFrame
    def _detectar_idioma_en_fila(texto):
        if pd.isna(texto) or not isinstance(texto, str) or len(texto.strip()) == 0:
            return None, None
        try:
            # El pipeline devuelve una lista de diccionarios, tomamos el primero
            resultado = classifier(texto)[0]
            idioma = resultado['label']
            confianza = resultado['score']
            return idioma, confianza
        except Exception as e:
            print(f"Error al procesar el texto: '{texto[:50]}...'. Error: {e}")
            return None, None

    newName = 'language_detected'
    # Aplicar la función a la columna especificada y crear nuevas columnas
    df[[newName, 'confianza_idioma']] = df[columna_texto].apply(
        lambda x: pd.Series(_detectar_idioma_en_fila(x))
    )

    return df

# --- Ejemplo de Uso ---
if __name__ == "__main__":
    # Crear un DataFrame de ejemplo
    data = {
        'id': [1, 2, 3, 4, 5, 6],
        'texto': [
            "Hello, how are you?",
            "Hola, ¿cómo estás?",
            "Je m'appelle Jean.",
            "Das ist ein deutsches Satz.",
            "こんにちは、お元気ですか？",
            "This is a mixed sentence with some Spanish words like 'hola'."
        ]
    }
    df_ejemplo = pd.DataFrame(data)

    print("DataFrame original:")
    print(df_ejemplo)
    print("-" * 30)

    # Detectar el idioma
    df_resultado = detectar_idioma_df(df_ejemplo, 'texto')

    print("\nDataFrame con idioma detectado:")
    print(df_resultado)

    # Ejemplo con otro modelo (si quieres probar)
    # try:
    #     df_resultado_alt = detectar_idioma_df(df_ejemplo.copy(), 'texto', modelo_nombre="facebook/fasttext-lid-176b") # Este modelo no funciona directamente con el pipeline de text-classification
    #     print("\nDataFrame con idioma detectado (FastText - si es compatible con pipeline):")
    #     print(df_resultado_alt)
    # except Exception as e:
    #     print(f"\nNo se pudo probar con FastText directamente con el pipeline de text-classification: {e}")

    # Ejemplo con texto vacío o nulo
    data_nulos = {
        'id': [7, 8, 9],
        'texto': [
            "    ",
            None,
            "Una frase corta."
        ]
    }
    df_nulos = pd.DataFrame(data_nulos)
    print("\nDataFrame con nulos/vacíos:")
    print(df_nulos)
    df_resultado_nulos = detectar_idioma_df(df_nulos, 'texto')
    print("\nDataFrame con nulos/vacíos y idioma detectado:")
    print(df_resultado_nulos)

DataFrame original:
   id                                              texto
0   1                                Hello, how are you?
1   2                                 Hola, ¿cómo estás?
2   3                                 Je m'appelle Jean.
3   4                        Das ist ein deutsches Satz.
4   5                                      こんにちは、お元気ですか？
5   6  This is a mixed sentence with some Spanish wor...
------------------------------

DataFrame con idioma detectado:
   id                                              texto language_detected  \
0   1                                Hello, how are you?                en   
1   2                                 Hola, ¿cómo estás?                es   
2   3                                 Je m'appelle Jean.                fr   
3   4                        Das ist ein deutsches Satz.                de   
4   5                                      こんにちは、お元気ですか？                ja   
5   6  This is a mixed sentence with some Spanish

In [18]:
import google.generativeai as genai
import nltk  # For sentence or token splitting
from typing import List

# Replace with your actual API key

def chunk_text(text: str, chunk_size: int = 500, chunk_type: str = "sentence") -> List[str]:
    """Splits text into chunks based on sentences, tokens, or paragraphs."""
    if chunk_type == "sentence":
        nltk.download('punkt')  # Make sure the punkt tokenizer is downloaded
        sentences = nltk.sent_tokenize(text)
        chunks = []
        current_chunk = ""
        for sentence in sentences:
            if len(current_chunk) + len(sentence) + 1 <= chunk_size:
                current_chunk += (sentence if not current_chunk else " " + sentence)
            else:
                chunks.append(current_chunk)
                current_chunk = sentence
        if current_chunk:
            chunks.append(current_chunk)
        return chunks

    elif chunk_type == "token":  # Example based on token count
        nltk.download('punkt')
        tokens = nltk.word_tokenize(text)
        chunks = []
        current_chunk = []
        current_len = 0
        for token in tokens:
            if current_len + 1 <= chunk_size:
                current_chunk.append(token)
                current_len += 1
            else:
                chunks.append(" ".join(current_chunk))
                current_chunk = [token]
                current_len = 1
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        return chunks
    # Add a paragraph based chunker if appropriate for the data format

    else:
        raise ValueError("Invalid chunk type. Choose 'sentence' or 'token'.")



def assign_categories(chunks: List[str], categories: List[str]) -> List[dict]:
    """Assigns categories to text chunks using the Gemini model."""
    results = []
    for chunk in chunks:
        prompt = f"""Given the following social media text chunk and a list of categories, 
        select the most relevant category or categories (if multiple are applicable) from the list. 
        Return your answer as a simple python list. ONLY RETURN THE LIST.
        Categories: {categories}
        Text Chunk: {chunk}"""
        response = model.generate_content(prompt)
        try:
            assigned_cats = eval(response.text)  
            results.append({"chunk": chunk, "categories": assigned_cats})
        except: 
            results.append({"chunk": chunk, "categories": ["UNKNOWN"]})
    return results

# --- Example Usage ---
# Replace with your large social media text
large_text = """This is a long example text representing social media posts. 
It might include discussions about product features, customer support interactions, 
and general brand mentions.  Some posts might be very positive, others negative, 
and some neutral.  The text needs to be processed to understand the different 
themes and sentiments expressed."""  

# Your initial categories
your_categories = ["Product Feedback", "Customer Support", "Brand Mentions", "Feature Requests"]


# Split the large text
text_chunks = chunk_text(large_text, chunk_size=200, 
                         chunk_type="token") 

# Assign categories to chunks
chunk_category_assignments = assign_categories(text_chunks, your_categories)

# Process the categorized chunks.
for item in chunk_category_assignments:
  print(f"Chunk: {item['chunk'][:50]}...")  
  print(f"Assigned Categories: {item['categories']}")
  print("-" * 20)


[nltk_data] Downloading package punkt to
[nltk_data]     E:\Users\1167486\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Chunk: This is a long example text representing social me...
Assigned Categories: ['Product Feedback', 'Customer Support', 'Brand Mentions']
--------------------


## Analizando redes para Divisas

In [19]:
import pandas as pd
import numpy as np

In [20]:
mainPath = 'E:/Users/1167486/Local/proyectos/divisas/Scraper_divisas'
XFile = mainPath + '/x_scraped_data_divisas.csv'
FbFile = mainPath + '/facebook_scraped_data_divisas.csv'
InstaFile = mainPath + '/instagram_scraped_data_divisas.csv'
TikTokFile = mainPath + '/tiktok_scraped_data_divisas.csv'
PlayStoreFile = mainPath + '/playstore_scraped_data_divisas.csv'

In [21]:
xData = pd.read_csv(XFile)
xData['social_media'] = 'X'
xData['name'] = ''

FbData = pd.read_csv(FbFile)
FbData['social_media'] = 'Facebook'
FbData['username'] = ''
FbData['name'] = ''
FbData['timestamp'] = ''

InstaData = pd.read_csv(InstaFile, encoding='latin1')
InstaData['social_media'] = 'Instagram'
TikTokData = pd.read_csv(TikTokFile)
TikTokData['social_media'] = 'TikTok'
TikTokData['name'] = ''
TikTokData['app'] = 'Wise'


PlayData = pd.read_csv(PlayStoreFile)
PlayData['social_media'] = 'Playstore'
PlayData['name'] = ''


In [22]:
print(f'Columnas de X : {xData.columns}')
print(f'Columnas de Fb : {FbData.columns}')
print(f'Columnas de Instagram : {InstaData.columns}')
print(f'Columnas de TikTok : {TikTokData.columns}')
print(f'Columnas de Playstore : {PlayData.columns}')

Columnas de X : Index(['tweet_text', 'username', 'timestamp', 'tweet_id', 'original_url',
       'tweet_link', 'app', 'original_tweet_link', 'original_social_media',
       'social_media', 'name'],
      dtype='object')
Columnas de Fb : Index(['comment', 'app', 'original_facebook_link', 'original_social_media',
       'social_media', 'username', 'name', 'timestamp'],
      dtype='object')
Columnas de Instagram : Index(['name', 'username', 'profile_id', 'date', 'likes', 'comment',
       'view_source', 'original_link', 'app', 'social_media'],
      dtype='object')
Columnas de TikTok : Index(['coment_id', 'user_id', 'username', 'user_alias', 'comment',
       'timestamp', 'likes', 'author_digged', 'answers', 'fixed', 'userpage',
       'social_media', 'name', 'app'],
      dtype='object')
Columnas de Playstore : Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion', 'sentiment

In [23]:
xData = xData[['original_tweet_link', 'username', 'name',  'tweet_text', 'timestamp', 'social_media', 'app']]
FbData=FbData[['original_facebook_link', 'username', 'name',  'comment', 'timestamp', 'social_media', 'app']]
InstaData=InstaData[['original_link', 'username',  'name', 'comment', 'date', 'social_media', 'app']]
TikTokData = TikTokData[['userpage', 'username', 'user_alias', 'comment', 'timestamp', 'social_media', 'app']]  
PlayData = PlayData[['reviewId', 'userName', 'name',  'content', 'at', 'social_media', 'app']]

generalCols= ['link_id', 'username', 'name',  'comment', 'timestamp', 'social_media', 'app']

xData.columns = generalCols 
FbData.columns = generalCols 
InstaData.columns = generalCols 
TikTokData.columns = generalCols 
PlayData.columns = generalCols 



In [24]:
socialMediaData = pd.concat([xData,  FbData, InstaData, TikTokData, PlayData], ignore_index=True)

In [25]:
socialMediaData.shape

(2943, 7)

In [26]:
print(socialMediaData.groupby(['social_media', 'app']).size().reset_index(name='counts'))

   social_media        app  counts
0      Facebook  DollarApp      19
1      Facebook       Wise     530
2     Instagram  DollarApp     185
3     Instagram    Revolut     261
4     Instagram       Wise     314
5     Playstore  DollarApp     300
6     Playstore    Revolut     300
7     Playstore       Wise     300
8        TikTok       Wise     124
9             X  DollarApp     156
10            X    Revolut     362
11            X       Wise      92


In [27]:
socialMediaData['username_to_gender'] = np.where(socialMediaData['social_media']!='Instagram', 
                                                 socialMediaData['username'], socialMediaData['name'])


In [28]:
socialMediaData['gender'] = socialMediaData['username_to_gender'].apply(predict_gender)

In [29]:
socialMediaData[socialMediaData['username_to_gender']=='']

,link_id,username,name,comment,timestamp,social_media,app,username_to_gender,gender
610,https://www.facebook.com/share/v/16cpnzsCdZ/,,,Si estás cansado de los cálculos interminables...,,Facebook,DollarApp,,Male
611,https://www.facebook.com/share/v/16cpnzsCdZ/,,,the card is available for Colombia ?,,Facebook,DollarApp,,Male
612,https://www.facebook.com/share/v/16cpnzsCdZ/,,,No me llega el pago que envié,,Facebook,DollarApp,,Male
613,https://www.facebook.com/share/1C47ivC3PV/,,,"Hola Gustavo, tuvimos un problema ayer por la ...",,Facebook,DollarApp,,Male
614,https://www.facebook.com/share/1C47ivC3PV/,,,Habiliten que se puedan usar los datos de cuen...,,Facebook,DollarApp,,Male
...,...,...,...,...,...,...,...,...,...
1154,https://www.facebook.com/share/p/16YREQCjQF/,,,me too,,Facebook,Wise,,Male
1155,https://www.facebook.com/share/p/16YREQCjQF/,,,Absolutely shocking news as I have been using ...,,Facebook,Wise,,Male
1156,https://www.facebook.com/share/p/16YREQCjQF/,,,It's very upsetting how you changed your regul...,,Facebook,Wise,,Male
1157,https://www.facebook.com/share/p/16YREQCjQF/,,,"Oh, well TOUGH - should NOT have closed my acc...",,Facebook,Wise,,Male


In [30]:
socialMediaData['gender'] = np.where((socialMediaData['username_to_gender'].isna()) |(socialMediaData['username_to_gender']==''), 'Unknown', socialMediaData['gender'])

In [31]:
print(socialMediaData.groupby([ 'app', 'gender']).size().reset_index(name='counts'))

         app   gender  counts
0  DollarApp   Female     290
1  DollarApp     Male     351
2  DollarApp  Unknown      19
3    Revolut   Female     367
4    Revolut     Male     556
5       Wise   Female     226
6       Wise     Male     604
7       Wise  Unknown     530


In [32]:
socialMediaData.head(5)

,link_id,username,name,comment,timestamp,social_media,app,username_to_gender,gender
0,https://x.com/RevolutApp/status/19153142986219...,@tawnyowlhillin1,,BBC News - Revolut: 'I was careful and followe...,2025-04-24T08:04:43.000Z,X,Revolut,@tawnyowlhillin1,Female
1,https://x.com/RevolutApp/status/19153142986219...,@denisPashchenko,,With banking licence or like crypto payment sy...,2025-04-24T08:36:05.000Z,X,Revolut,@denisPashchenko,Male
2,https://x.com/RevolutApp/status/19153142986219...,@TimChaney598055,,WTF is happening with your app in Spain? I can...,2025-05-11T08:40:46.000Z,X,Revolut,@TimChaney598055,Female
3,https://x.com/RevolutApp/status/19153142986219...,@rach3kids,,"Hello, I am trying to raise a complaint, but t...",2025-05-02T08:30:10.000Z,X,Revolut,@rach3kids,Male
4,https://x.com/RevolutApp/status/19153142986219...,@GameofLife_ok,,Tengo mi cuenta bloqueada desde hace días sin ...,2025-04-25T16:01:52.000Z,X,Revolut,@GameofLife_ok,Male


In [36]:
#socialMediaData['language_detected'] = socialMediaData['comment'].apply(lambda x: GoogleTranslator(source='auto', target='es').detect(x).lang)

socialMediaData = detectar_idioma_df(socialMediaData, 'comment')

In [37]:
socialMediaData.head(5)

,link_id,username,name,comment,timestamp,social_media,app,username_to_gender,gender,comment_es,confianza_idioma,language_detected
0,https://x.com/RevolutApp/status/19153142986219...,@tawnyowlhillin1,,BBC News - Revolut: 'I was careful and followe...,2025-04-24T08:04:43.000Z,X,Revolut,@tawnyowlhillin1,Female,en,0.959597,en
1,https://x.com/RevolutApp/status/19153142986219...,@denisPashchenko,,With banking licence or like crypto payment sy...,2025-04-24T08:36:05.000Z,X,Revolut,@denisPashchenko,Male,en,0.915091,en
2,https://x.com/RevolutApp/status/19153142986219...,@TimChaney598055,,WTF is happening with your app in Spain? I can...,2025-05-11T08:40:46.000Z,X,Revolut,@TimChaney598055,Female,en,0.994166,en
3,https://x.com/RevolutApp/status/19153142986219...,@rach3kids,,"Hello, I am trying to raise a complaint, but t...",2025-05-02T08:30:10.000Z,X,Revolut,@rach3kids,Male,en,0.993253,en
4,https://x.com/RevolutApp/status/19153142986219...,@GameofLife_ok,,Tengo mi cuenta bloqueada desde hace días sin ...,2025-04-25T16:01:52.000Z,X,Revolut,@GameofLife_ok,Male,es,0.993852,es


In [39]:
print(socialMediaData.groupby([ 'app', 'language_detected']).size().reset_index(name='counts'))

          app language_detected  counts
0   DollarApp                bg       2
1   DollarApp                el       9
2   DollarApp                en      46
3   DollarApp                es     485
4   DollarApp                fr       3
5   DollarApp                hi      21
6   DollarApp                it      14
7   DollarApp                pt      56
8   DollarApp                sw       6
9   DollarApp                tr       3
10  DollarApp                ur      13
11    Revolut                bg       4
12    Revolut                de       4
13    Revolut                en     519
14    Revolut                es     242
15    Revolut                fr      12
16    Revolut                hi      33
17    Revolut                it      17
18    Revolut                nl       5
19    Revolut                pl       4
20    Revolut                pt      27
21    Revolut                ru       2
22    Revolut                sw      23
23    Revolut                tr       4


In [53]:
dictSupported = {'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'assamese': 'as', 'aymara': 'ay', 'azerbaijani': 'az', 'bambara': 'bm', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bhojpuri': 'bho', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-CN', 'chinese (traditional)': 'zh-TW', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dhivehi': 'dv', 'dogri': 'doi', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'ewe': 'ee', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'guarani': 'gn', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'iw', 'hindi': 'hi', 'hmong': 'hmn', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'ilocano': 'ilo', 'indonesian': 'id', 'irish': 'ga', 'italian': 'it', 'japanese': 'ja', 'javanese': 'jw', 'kannada': 'kn', 'kazakh': 'kk', 'khmer': 'km', 'kinyarwanda': 'rw', 'konkani': 'gom', 'korean': 'ko', 'krio': 'kri', 'kurdish (kurmanji)': 'ku', 'kurdish (sorani)': 'ckb', 'kyrgyz': 'ky', 'lao': 'lo', 'latin': 'la', 'latvian': 'lv', 'lingala': 'ln', 'lithuanian': 'lt', 'luganda': 'lg', 'luxembourgish': 'lb', 'macedonian': 'mk', 'maithili': 'mai', 'malagasy': 'mg', 'malay': 'ms', 'malayalam': 'ml', 'maltese': 'mt', 'maori': 'mi', 'marathi': 'mr', 'meiteilon (manipuri)': 'mni-Mtei', 'mizo': 'lus', 'mongolian': 'mn', 'myanmar': 'my', 'nepali': 'ne', 'norwegian': 'no', 'odia (oriya)': 'or', 'oromo': 'om', 'pashto': 'ps', 'persian': 'fa', 'polish': 'pl', 'portuguese': 'pt', 'punjabi': 'pa', 'quechua': 'qu', 'romanian': 'ro', 'russian': 'ru', 'samoan': 'sm', 'sanskrit': 'sa', 'scots gaelic': 'gd', 'sepedi': 'nso', 'serbian': 'sr', 'sesotho': 'st', 'shona': 'sn', 'sindhi': 'sd', 'sinhala': 'si', 'slovak': 'sk', 'slovenian': 'sl', 'somali': 'so', 'spanish': 'es', 'sundanese': 'su', 'swahili': 'sw', 'swedish': 'sv', 'tajik': 'tg', 'tamil': 'ta', 'tatar': 'tt', 'telugu': 'te', 'thai': 'th', 'tigrinya': 'ti', 'tsonga': 'ts', 'turkish': 'tr', 'turkmen': 'tk', 'twi': 'ak', 'ukrainian': 'uk', 'urdu': 'ur', 'uyghur': 'ug', 'uzbek': 'uz', 'vietnamese': 'vi', 'welsh': 'cy', 'xhosa': 'xh', 'yiddish': 'yi', 'yoruba': 'yo', 'zulu': 'zu'}
supportedLanguages = list(dictSupported.values())

In [54]:
'''
socialMediaData['comment_es'] = socialMediaData.apply(
    lambda x: (
        # If x['language_detected'] is truthy (e.g., not None, not an empty string), proceed to translate
        (translated := TranslateText(x['comment'], source=x['language_detected'], target='es'),
         x['comment'] if translated is None  else translated)[1] # Use original comment if translation is None
        if x.get('language_detected')  # Check if language is detected
        else '' # Fallback to original comment if language_detected is falsy
    ), axis=1
)
'''

socialMediaData['comment_es'] = socialMediaData.apply(lambda x: translate_comment_safely(x, target_language='es', SUPPORTED_LANGUAGES=supportedLanguages), axis=1)

In [56]:
socialMediaData['comment_es'] = np.where(~socialMediaData['language_detected'].isin( supportedLanguages), '',
                                          socialMediaData['comment_es'])

In [57]:
socialMediaData['sentiment'] = socialMediaData['comment_es'].apply(lambda x: generate_sentiment_analysis(x, analizer=sentiments_analizer))
socialMediaData['emotion'] = socialMediaData['comment_es'].apply(lambda x: generate_emotions_analysis(x, analizer=emotions_analizer))



SpacyTextBlob polarity (0.000) is neutral/weak. Consulting Pysentimiento for: 'BBC News - Revolut: "Tuve cuidado y seguí las instrucciones ...'
Pysentimiento también dice que es neutral: {'NEG': 0.10693790018558502, 'NEU': 0.8126563429832458, 'POS': 0.08040570467710495}
SpacyTextBlob polarity (0.000) is neutral/weak. Consulting Pysentimiento for: '¿Con licencia bancaria o como sistema de pago criptográfico?...'
Pysentimiento valores: Negativo (Probas: {'NEG': 0.8201983571052551, 'NEU': 0.17586643993854523, 'POS': 0.0039352960884571075})
SpacyTextBlob polarity (0.000) is neutral/weak. Consulting Pysentimiento for: '¿WTF está sucediendo con su aplicación en España? Ya no pued...'
Pysentimiento valores: Negativo (Probas: {'NEG': 0.9363893866539001, 'NEU': 0.06115547940135002, 'POS': 0.002455150242894888})
SpacyTextBlob polarity (0.000) is neutral/weak. Consulting Pysentimiento for: 'Hola, estoy tratando de presentar una queja, pero la aplicac...'
Pysentimiento valores: Negativo (Probas: {

In [58]:
socialMediaData.head(5)

,link_id,username,name,comment,timestamp,social_media,app,username_to_gender,gender,comment_es,confianza_idioma,language_detected,sentiment,emotion
0,https://x.com/RevolutApp/status/19153142986219...,@tawnyowlhillin1,,BBC News - Revolut: 'I was careful and followe...,2025-04-24T08:04:43.000Z,X,Revolut,@tawnyowlhillin1,Female,"BBC News - Revolut: ""Tuve cuidado y seguí las ...",0.959597,en,Neutral,Neutral
1,https://x.com/RevolutApp/status/19153142986219...,@denisPashchenko,,With banking licence or like crypto payment sy...,2025-04-24T08:36:05.000Z,X,Revolut,@denisPashchenko,Male,¿Con licencia bancaria o como sistema de pago ...,0.915091,en,Negativo,Neutral
2,https://x.com/RevolutApp/status/19153142986219...,@TimChaney598055,,WTF is happening with your app in Spain? I can...,2025-05-11T08:40:46.000Z,X,Revolut,@TimChaney598055,Female,¿WTF está sucediendo con su aplicación en Espa...,0.994166,en,Negativo,Neutral
3,https://x.com/RevolutApp/status/19153142986219...,@rach3kids,,"Hello, I am trying to raise a complaint, but t...",2025-05-02T08:30:10.000Z,X,Revolut,@rach3kids,Male,"Hola, estoy tratando de presentar una queja, p...",0.993253,en,Negativo,Neutral
4,https://x.com/RevolutApp/status/19153142986219...,@GameofLife_ok,,Tengo mi cuenta bloqueada desde hace días sin ...,2025-04-25T16:01:52.000Z,X,Revolut,@GameofLife_ok,Male,Tengo mi cuenta bloqueada desde hace días sin ...,0.993852,es,Positivo,Enojo


In [77]:
import google.generativeai as genai
import nltk  # For sentence or token splitting
from typing import List

# Replace with your actual API key

def chunk_text(text: str, chunk_size: int = 500, chunk_type: str = "sentence") -> List[str]:
    """Splits text into chunks based on sentences, tokens, or paragraphs."""
    if chunk_type == "sentence":
        nltk.download('punkt')  # Make sure the punkt tokenizer is downloaded
        sentences = nltk.sent_tokenize(text)
        chunks = []
        current_chunk = ""
        for sentence in sentences:
            if len(current_chunk) + len(sentence) + 1 <= chunk_size:
                current_chunk += (sentence if not current_chunk else " " + sentence)
            else:
                chunks.append(current_chunk)
                current_chunk = sentence
        if current_chunk:
            chunks.append(current_chunk)
        return chunks

    elif chunk_type == "token":  # Example based on token count
        nltk.download('punkt')
        tokens = nltk.word_tokenize(text)
        chunks = []
        current_chunk = []
        current_len = 0
        for token in tokens:
            if current_len + 1 <= chunk_size:
                current_chunk.append(token)
                current_len += 1
            else:
                chunks.append(" ".join(current_chunk))
                current_chunk = [token]
                current_len = 1
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        return chunks
    # Add a paragraph based chunker if appropriate for the data format

    else:
        raise ValueError("Invalid chunk type. Choose 'sentence' or 'token'.")



def assign_categories(chunks: List[str], categories: List[str]) -> List[dict]:
    """Assigns categories to text chunks using the Gemini model."""
    results = []
    import time 
    for chunk in chunks:
        time.sleep(20)
        prompt = f"""
        Dados los siguientes fragmentos de texto de redes sociales y una lista de categorías,
        selecciona la categoría más relevante o categorías (si son múltiples) de la lista.
        Devuelve tu respuesta como una lista de Python simple. SOLO DEVUELVE LA LISTA.
        Categorías: {categories}
        Fragmento de texto: {chunk}"""
        response = model.generate_content(prompt)
        try:
            assigned_cats = eval(response.text)  
            results.append({"chunk": chunk, "categories": assigned_cats})
        except: 
            results.append({"chunk": chunk, "categories": ["UNKNOWN"]})
    return results

In [ ]:
# --- Example Usage ---
# Replace with your large social media text
large_text = """This is a long example text representing social media posts. 
It might include discussions about product features, customer support interactions, 
and general brand mentions.  Some posts might be very positive, others negative, 
and some neutral.  The text needs to be processed to understand the different 
themes and sentiments expressed."""  


### Get the main categories from the prompt
generalCommentsinitial = ' '.join(socialMediaData[socialMediaData['social_media'] == 'Playstore']['comment_es'])




In [60]:
prompt = f'''
Analiza el siguiente texto:
"{generalCommentsinitial}"

Tu tarea es extraer:
1. Funcionalidades clave (tanto positivas como negativas). Estas deben ser frases concisas, por ejemplo: 'App falla en depósitos', 'Interfaz intuitiva y rápida'.
2. Quince (15) categorías relevantes y sencillas (máximo 4 palabras cada una) que resuman los temas principales o aspectos mencionados en el texto.

IMPORTANTE: Formatea TODA tu respuesta EXCLUSIVAMENTE como una ÚNICA cadena de texto que represente una lista de Python.
Esta lista debe contener strings. Cada string puede ser una funcionalidad o una categoría.
Ejemplo de formato de respuesta esperado: ['funcionalidad A', 'funcionalidad B', 'categoría X', 'categoría Y', ..., 'categoría Z']
No incluyas NINGÚN texto, explicación, ni markdown (como ```python ... ```) antes o después de esta lista. Solo la lista en formato de cadena.
'''


In [61]:
original_category = model.generate_content(prompt)

In [63]:
original_category = original_category.text.strip()

In [65]:
mainLargeText = ' '.join(socialMediaData['comment_es'])


In [82]:
text_chunks = chunk_text(mainLargeText, chunk_size=7000, 
                         chunk_type="token") 



[nltk_data] Downloading package punkt to
[nltk_data]     E:\Users\1167486\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [83]:
print(len(text_chunks))

8


In [84]:
# Assign categories to chunks
chunk_category_assignments = assign_categories(text_chunks, original_category)



In [85]:
# Process the categorized chunks.
for item in chunk_category_assignments:
  print(f"Chunk: {item['chunk'][:50]}...")  
  print(f"Assigned Categories: {item['categories']}")
  print("-" * 20)


Chunk: BBC News - Revolut : `` Tuve cuidado y seguí las i...
Assigned Categories: ['UNKNOWN']
--------------------
Chunk: la gente nos pueda hacer envíos de USDc via Zelle ...
Assigned Categories: ['Problemas con depósitos', 'Servicio al cliente deficiente', 'Bloqueo de cuentas inesperado', 'Retiros problemáticos', 'Soporte tarda en responder', 'Seguridad cuestionable', 'Atención al cliente', 'Estabilidad de la aplicación', 'Tiempo de respuesta', 'Confianza y seguridad']
--------------------
Chunk: sigue cambiando ? Se suponía que debía recibir din...
Assigned Categories: ['Problemas con depósitos', 'Bloqueo de cuentas inesperado', 'Servicio al cliente deficiente', 'Retiros problemáticos', 'Soporte tarda en responder', 'Seguridad cuestionable', 'Atención al cliente', 'Seguridad de la app', 'Problemas con la tarjeta', 'Tiempo de respuesta', 'Confianza y seguridad', 'Disponibilidad de servicios', 'Estabilidad de la aplicación']
--------------------
Chunk: declaraciones de otros bancos , 

In [86]:
item['categories'] = [x for x in item['categories'] if x not in ['No aplica', '']]

In [91]:
categorias = item['categories'] 

In [92]:
categorias.append('No aplica')

In [93]:
categorias

['App intuitiva y fácil de usar',
 'Tipo de cambio conveniente',
 'Transferencias internacionales',
 'Cambio de divisas',
 'Problemas con la tarjeta',
 'Bloqueo de cuentas inesperado',
 'Servicio al cliente deficiente',
 'Comisiones elevadas',
 'Seguridad de la app',
 'Experiencia de usuario',
 'Funcionalidades de la app',
 'Tiempo de respuesta',
 'Estabilidad de la aplicación',
 'Sistema de verificación',
 'Confianza y seguridad',
 'Proceso de registro',
 'Transferencias rápidas y fáciles',
 'No aplica']

In [111]:
#generate_zero_shot_classification
#socialMediaData['categories'] = socialMediaData['comment_es'].apply(lambda x: assign_categories(x, categorias))
#test= socialMediaData.head(500)
#test['categories'] = test['comment_es'].apply(lambda x: generate_zero_shot_classification(x, categorias))
socialMediaData['categories'] = socialMediaData['comment_es'].apply(lambda x: generate_zero_shot_classification(x, categorias))

In [ ]:
#test.head(5)

,link_id,username,name,comment,timestamp,social_media,app,username_to_gender,gender,comment_es,confianza_idioma,language_detected,sentiment,emotion,categories
0,https://x.com/RevolutApp/status/19153142986219...,@tawnyowlhillin1,,BBC News - Revolut: 'I was careful and followe...,2025-04-24T08:04:43.000Z,X,Revolut,@tawnyowlhillin1,Female,"BBC News - Revolut: ""Tuve cuidado y seguí las ...",0.959597,en,Neutral,Neutral,Experiencia de usuario
1,https://x.com/RevolutApp/status/19153142986219...,@denisPashchenko,,With banking licence or like crypto payment sy...,2025-04-24T08:36:05.000Z,X,Revolut,@denisPashchenko,Male,¿Con licencia bancaria o como sistema de pago ...,0.915091,en,Negativo,Neutral,Bloqueo de cuentas inesperado
2,https://x.com/RevolutApp/status/19153142986219...,@TimChaney598055,,WTF is happening with your app in Spain? I can...,2025-05-11T08:40:46.000Z,X,Revolut,@TimChaney598055,Female,¿WTF está sucediendo con su aplicación en Espa...,0.994166,en,Negativo,Neutral,Experiencia de usuario
3,https://x.com/RevolutApp/status/19153142986219...,@rach3kids,,"Hello, I am trying to raise a complaint, but t...",2025-05-02T08:30:10.000Z,X,Revolut,@rach3kids,Male,"Hola, estoy tratando de presentar una queja, p...",0.993253,en,Negativo,Neutral,Experiencia de usuario
4,https://x.com/RevolutApp/status/19153142986219...,@GameofLife_ok,,Tengo mi cuenta bloqueada desde hace días sin ...,2025-04-25T16:01:52.000Z,X,Revolut,@GameofLife_ok,Male,Tengo mi cuenta bloqueada desde hace días sin ...,0.993852,es,Positivo,Enojo,Bloqueo de cuentas inesperado


In [114]:
#test.groupby(['categories']).size().reset_index(name='counts')
socialMediaData.groupby(['categories']).size().reset_index(name='counts')

,categories,counts
0,App intuitiva y fácil de usar,85
1,Bloqueo de cuentas inesperado,251
2,Cambio de divisas,10
3,Comisiones elevadas,82
4,Confianza y seguridad,344
5,Estabilidad de la aplicación,55
6,Experiencia de usuario,1286
7,Funcionalidades de la app,6
8,No aplica,204
9,Problemas con la tarjeta,136


In [129]:
socialMediaData.rename(columns={'gender': 'gender_backup'}, inplace=True)

In [130]:
#socialMediaData[['gender', 'gender_confidence', 'gender_liability']] = socialMediaData['username_to_gender'].apply(predict_gender_from_username)
gender_results_series = socialMediaData['username_to_gender'].apply(predict_gender_from_username)
expanded_gender_df = gender_results_series.apply(pd.Series)
socialMediaData = socialMediaData.join(expanded_gender_df)


In [131]:
socialMediaData.head(5)

,link_id,username,name,comment,timestamp,social_media,app,username_to_gender,gender_backup,comment_es,confianza_idioma,language_detected,sentiment,emotion,categories,gender,confidence,gender_liability
0,https://x.com/RevolutApp/status/19153142986219...,@tawnyowlhillin1,,BBC News - Revolut: 'I was careful and followe...,2025-04-24T08:04:43.000Z,X,Revolut,@tawnyowlhillin1,Female,"BBC News - Revolut: ""Tuve cuidado y seguí las ...",0.959597,en,Neutral,Neutral,Experiencia de usuario,Unknown,0.550350,Low confidence
1,https://x.com/RevolutApp/status/19153142986219...,@denisPashchenko,,With banking licence or like crypto payment sy...,2025-04-24T08:36:05.000Z,X,Revolut,@denisPashchenko,Male,¿Con licencia bancaria o como sistema de pago ...,0.915091,en,Negativo,Neutral,Bloqueo de cuentas inesperado,Unknown,0.666464,Low confidence
2,https://x.com/RevolutApp/status/19153142986219...,@TimChaney598055,,WTF is happening with your app in Spain? I can...,2025-05-11T08:40:46.000Z,X,Revolut,@TimChaney598055,Female,¿WTF está sucediendo con su aplicación en Espa...,0.994166,en,Negativo,Neutral,Experiencia de usuario,Unknown,0.648422,Low confidence
3,https://x.com/RevolutApp/status/19153142986219...,@rach3kids,,"Hello, I am trying to raise a complaint, but t...",2025-05-02T08:30:10.000Z,X,Revolut,@rach3kids,Male,"Hola, estoy tratando de presentar una queja, p...",0.993253,en,Negativo,Neutral,Experiencia de usuario,Male,0.832703,High confidence
4,https://x.com/RevolutApp/status/19153142986219...,@GameofLife_ok,,Tengo mi cuenta bloqueada desde hace días sin ...,2025-04-25T16:01:52.000Z,X,Revolut,@GameofLife_ok,Male,Tengo mi cuenta bloqueada desde hace días sin ...,0.993852,es,Positivo,Enojo,Bloqueo de cuentas inesperado,Male,0.777338,High confidence


In [132]:
socialMediaData['gender'] = np.where(socialMediaData['gender_liability'] =='Low confidence', 'Unknown', socialMediaData['gender'])

In [133]:
socialMediaData.drop(columns=['gender_backup'], inplace=True, errors='ignore')

In [134]:
socialMediaData.to_csv(mainPath + '/socialMediaDataDivisas.csv', index=False, 
                       encoding='utf-8')
#test.to_csv(mainPath + '/socialMediaDataDivisas.csv', index=False, encoding='utf-8')

## Iniciando ya con el dataframe categorizado inicialmente

In [7]:
import pandas as pd 
mainPath = 'E:/Users/1167486/Local/proyectos/divisas/Scraper_divisas'

socialMediaData = pd.read_csv(mainPath + '/socialMediaDataDivisas.csv', encoding='utf-8')

In [8]:
def promptToCategories(text, categories, prompt):
    prompt = prompt  + f"""Categorías: {categories}
    Fragmento de texto: {text}"""
    response = model.generate_content(prompt)
    try:
        assigned_cats = eval(response.text)  
        return assigned_cats
    except: 
        return ["UNKNOWN"]

In [37]:
def get_verbs(text: str) -> list[str]:
    """
    Extracts and lemmatizes verbs from a given text.

    Args:
        text: The input string to process.

    Returns:
        A list of unique lemmatized verbs found in the text.
        Returns an empty list if no verbs are found or the text is empty.
    """
    if not text:
        return []
    doc = nlp_model(text)
    verbs = [token.lemma_.lower() for token in doc if token.pos_ == "VERB"]
    # Return unique verbs to avoid duplicates if a verb appears multiple times
    return sorted(list(set(verbs)))

def get_nouns(text: str) -> list[str]:
    """
    Extracts and lemmatizes nouns (common and proper) from a given text.

    Args:
        text: The input string to process.

    Returns:
        A list of unique lemmatized nouns found in the text.
        Returns an empty list if no nouns are found or the text is empty.
    """
    if not text:
        return []
    doc = nlp_model(text)
    # We'll consider both common nouns (NOUN) and proper nouns (PROPN)
    nouns = [token.lemma_.lower() for token in doc if token.pos_ in ("NOUN", "PROPN")]
    # Return unique nouns
    return sorted(list(set(nouns)))

In [45]:
def get_semantic_main_verb_es(text: str) -> str | None:
    """
    Attempts to identify the verb that carries the main semantic meaning in a Spanish phrase.
    It prioritizes verbs that are part of an open clausal complement (xcomp),
    especially the "deepest" one in a chain (e.g., in "querer intentar hacer", "hacer" would be preferred).
    If no such xcomp is found, it falls back to the root verb of the sentence.

    Args:
        text: The input Spanish string.

    Returns:
        The lemmatized main semantic verb (lowercase), or None if no verb is found
        or the text is empty.
    """
    if not text or not text.strip():
        return None

    doc = nlp_model(text)

    candidate_verb_token = None
    root_verb_token = None

    # 1. Find the root of the sentence, if it's a verb or auxiliary.
    # A sentence typically has one root.
    for token in doc:
        if token.dep_ == "ROOT":
            if token.pos_ in ("VERB", "AUX"): # e.g., main verb or copula like 'ser'
                root_verb_token = token
            break # Found the root, no need to look further for it.

    # 2. Look for an open clausal complement (xcomp) that is a verb
    #    and whose head is also a verb. This often captures the main action
    #    in constructions like "voy a comer", "quiero hacer", "decidí intentar trabajar".
    #    If there's a chain of xcomps, the last one is often the most specific action.
    xcomp_candidate = None
    for token in doc:
        if token.pos_ == "VERB" and token.dep_ == "xcomp":
            # Check if its head is a verb (or aux, as aux can head xcomps too)
            if token.head.pos_ in ("VERB", "AUX"):
                xcomp_candidate = token # Keep updating; the last one in sentence order will be taken.
                                        # This tends to get the "deepest" verb in a chain like
                                        # "decidí (head) -> intentar (xcomp, head) -> trabajar (xcomp)"

    # 3. Prioritize the xcomp candidate if found. Otherwise, use the root verb.
    if xcomp_candidate:
        candidate_verb_token = xcomp_candidate
    elif root_verb_token:
        candidate_verb_token = root_verb_token
    else:
        # Fallback: if no suitable root or xcomp was identified (e.g., unusual structure),
        # try to find the first verb in the sentence.
        for token in doc:
            if token.pos_ == "VERB":
                candidate_verb_token = token
                break
                
    if candidate_verb_token:
        return candidate_verb_token.lemma_.lower()
    
    return None


In [11]:
socialMediaData['comment_es'] = socialMediaData['comment_es'].astype(str)

In [12]:
generalCommentsinitial = ' '.join(socialMediaData[socialMediaData['social_media'] == 'Playstore']['comment_es'])


In [13]:
import ast 
promptTiempoIncomodidad = f'''
Analiza el siguiente texto:
"{generalCommentsinitial}"

Tu tarea es extraer:
1. Máximo 4 categorías sobre el uso de la app que involucren temáticas como pérdida de tiempo o incomodidad en su uso  

IMPORTANTE: Formatea TODA tu respuesta EXCLUSIVAMENTE como una ÚNICA cadena de texto que represente una lista de Python.
Esta lista debe contener strings. Cada string puede ser una funcionalidad o una categoría.
Ejemplo de formato de respuesta esperado: ['funcionalidad A', 'funcionalidad B', 'categoría X', 'categoría Y', ..., 'categoría Z']
No incluyas NINGÚN texto, explicación, ni markdown (como ```python ... ```) antes o después de esta lista. Solo la lista en formato de cadena.
'''

categoriesTime = model.generate_content(promptTiempoIncomodidad)
categoriesTime = categoriesTime.text.strip()
categoriesTime = ast.literal_eval(categoriesTime)
categoriesTime.append('No aplica')
print(categoriesTime)

['Problemas con la creación de cuentas y verificación de identidad', 'Bloqueo de cuentas sin explicación', 'Problemas con el servicio de atención al cliente y tiempos de respuesta', 'Comisiones y cargos inesperados', 'No aplica']


In [ ]:
#categoriesTime = ['Problemas con la creación de cuentas y verificación de identidad', 'Bloqueo de cuentas sin explicación', 'Atención al cliente deficiente o inexistente', 'Problemas con las transferencias y recepción de fondos']


In [ ]:

#socialMediaData['categories'] = socialMediaData['comment_es'].apply(lambda x: generate_zero_shot_classification(x, categorias))

In [14]:
promptSegundosPasos= f'''
Analiza el siguiente texto:
"{generalCommentsinitial}"

Tu tarea es extraer:
1. Máximo 4 categorías sobre el uso de la app que involucren temáticas sobe pasos extras al usar la app o características que no son intuitivas o que hagan que el usuario use un plan B u otra app
Las respuestas deben ser frases cortas de máximo 4 palabras

IMPORTANTE: Formatea TODA tu respuesta EXCLUSIVAMENTE como una ÚNICA cadena de texto que represente una lista de Python.
Esta lista debe contener strings. Cada string puede ser una funcionalidad o una categoría.
Ejemplo de formato de respuesta esperado: ['funcionalidad A', 'funcionalidad B', 'categoría X', 'categoría Y', ..., 'categoría Z']
No incluyas NINGÚN texto, explicación, ni markdown (como ```python ... ```) antes o después de esta lista. Solo la lista en formato de cadena.
'''

categoriesSecondSteps= model.generate_content(promptSegundosPasos)
categoriesSecondSteps = categoriesSecondSteps.text.strip()
categoriesSecondSteps = ast.literal_eval(categoriesSecondSteps)

categoriesSecondSteps.append('No aplica')
print(categoriesSecondSteps)


['Verificación compleja', 'Soporte deficiente', 'Transferencias lentas', 'Tarifas elevadas', 'No aplica']


In [16]:
import time
time.sleep(120)
promptCaracteristicasDeseadas= f'''
Analiza el siguiente texto:
"{generalCommentsinitial}"

Tu tarea es extraer:
1. Máximo 10 categorías sobre las características más queridas por los usuarios y que realmente aporten valor
Las respuestas deben ser frases cortas de máximo 4 palabras

IMPORTANTE: Formatea TODA tu respuesta EXCLUSIVAMENTE como una ÚNICA cadena de texto que represente una lista de Python.
Esta lista debe contener strings. Cada string puede ser una funcionalidad o una categoría.
Ejemplo de formato de respuesta esperado: ['funcionalidad A', 'funcionalidad B', 'categoría X', 'categoría Y', ..., 'categoría Z']
No incluyas NINGÚN texto, explicación, ni markdown (como ```python ... ```) antes o después de esta lista. Solo la lista en formato de cadena.
'''
categoriesDesired= model.generate_content(promptCaracteristicasDeseadas)
categoriesDesired = categoriesDesired.text.strip()
categoriesDesired = ast.literal_eval(categoriesDesired)

categoriesDesired.append('No aplica')

print(categoriesDesired)


['Transferencias rápidas', 'Tipo cambio bueno', 'Fácil de usar', 'Pagos internacionales', 'Comisiones bajas', 'Seguridad', 'Tarjeta virtual', 'Soporte rápido', 'Confiable', 'App eficiente', 'No aplica']


In [18]:
time.sleep(90)
promptAtencion= f'''
Analiza el siguiente texto:
"{generalCommentsinitial}"

Tu tarea es extraer:
1. Máximo 4 categorías sobre la atención al cliente, como tiempos de respuesta, calidad de la atención, etc.
Las respuestas deben ser frases cortas de máximo 4 palabras

IMPORTANTE: Formatea TODA tu respuesta EXCLUSIVAMENTE como una ÚNICA cadena de texto que represente una lista de Python.
Esta lista debe contener strings. Cada string puede ser una funcionalidad o una categoría.
Ejemplo de formato de respuesta esperado: ['funcionalidad A', 'funcionalidad B', 'categoría X', 'categoría Y', ..., 'categoría Z']
No incluyas NINGÚN texto, explicación, ni markdown (como ```python ... ```) antes o después de esta lista. Solo la lista en formato de cadena.
'''
categoriesAttention= model.generate_content(promptAtencion)
categoriesAttention = categoriesAttention.text.strip()
categoriesAttention = ast.literal_eval(categoriesAttention)

categoriesAttention.append('No aplica')

print(categoriesAttention)


['Tiempos de respuesta lentos', 'Atención al cliente pésima', 'Soporte no responde', 'Publicidad engañosa', 'No aplica']


In [ ]:
#categoriesTime = ['Problemas con la creación de cuentas y verificación de identidad', 'Bloqueo de cuentas sin explicación', 'Atención al cliente deficiente o inexistente', 'Problemas con transferencias y pagos: retrasos, errores, comisiones excesivas', 'No aplica']
#categoriesSecondSteps = ['Verificación de identidad compleja', 'Soporte al cliente deficiente', 'Proceso de registro difícil', 'Comisiones elevadas', 'No aplica']
#categoriesDesired = ['Transferencias rápidas', 'Cambios favorables', 'Fácil de usar', 'Seguridad confiable', 'No aplica']
#categoriesAttention = ['Tiempos de respuesta lentos', 'Atención al cliente pésima', 'Soporte no responde', 'Servicio cliente deficiente', 'No aplica']


In [19]:
socialMediaData['categories_time'] = socialMediaData['comment_es'].apply(lambda x: generate_zero_shot_classification(x, categoriesTime))

In [20]:
socialMediaData['categories_second_steps'] = socialMediaData['comment_es'].apply(lambda x: generate_zero_shot_classification(x, categoriesSecondSteps))

In [21]:
socialMediaData['categories_desired_categories'] = socialMediaData['comment_es'].apply(lambda x: generate_zero_shot_classification(x, categoriesDesired))

In [22]:
socialMediaData['categories_client_attention'] = socialMediaData['comment_es'].apply(lambda x: generate_zero_shot_classification(x, categoriesAttention))

In [23]:
socialMediaData.to_csv(mainPath + '/socialMediaDataDivisas.csv', index=False, 
                       encoding='utf-8')

In [24]:
socialMediaData.groupby(['categories_time']).size().reset_index(name='counts')

,categories_time,counts
0,Bloqueo de cuentas sin explicación,448
1,Comisiones y cargos inesperados,1444
2,No aplica,270
3,Problemas con el servicio de atención al clien...,619
4,Problemas con la creación de cuentas y verific...,162


In [25]:
socialMediaData.groupby(['categories_second_steps']).size().reset_index(name='counts')

,categories_second_steps,counts
0,No aplica,333
1,Soporte deficiente,841
2,Tarifas elevadas,443
3,Transferencias lentas,193
4,Verificación compleja,1133


In [26]:
socialMediaData.groupby(['categories_desired_categories']).size().reset_index(name='counts')

,categories_desired_categories,counts
0,App eficiente,221
1,Comisiones bajas,97
2,Confiable,848
3,Fácil de usar,326
4,No aplica,311
5,Pagos internacionales,119
6,Seguridad,281
7,Soporte rápido,96
8,Tarjeta virtual,43
9,Tipo cambio bueno,301


In [38]:
socialMediaData['verbs'] = socialMediaData['comment_es'].apply(get_verbs)
socialMediaData['nouns'] = socialMediaData['comment_es'].apply(get_nouns)

In [46]:
socialMediaData['main_verb'] = socialMediaData['comment_es'].apply(get_semantic_main_verb_es)

In [47]:
socialMediaData.head(10)

,link_id,username,name,comment,timestamp,social_media,app,username_to_gender,comment_es,confianza_idioma,...,gender,confidence,gender_liability,categories_time,categories_second_steps,categories_desired_categories,categories_client_attention,verbs,nouns,main_verb
0,https://x.com/RevolutApp/status/19153142986219...,@tawnyowlhillin1,NaN,BBC News - Revolut: 'I was careful and followe...,2025-04-24T08:04:43.000Z,X,Revolut,@tawnyowlhillin1,"BBC News - Revolut: ""Tuve cuidado y seguí las ...",0.959597,...,Unknown,0.550350,Low confidence,Problemas con la creación de cuentas y verific...,Soporte deficiente,Soporte rápido,Soporte no responde,"[perder, seguí, tener]","[bbc, criptografía, cuidado, instrucción, news...",tener
1,https://x.com/RevolutApp/status/19153142986219...,@denisPashchenko,NaN,With banking licence or like crypto payment sy...,2025-04-24T08:36:05.000Z,X,Revolut,@denisPashchenko,¿Con licencia bancaria o como sistema de pago ...,0.915091,...,Unknown,0.666464,Low confidence,Bloqueo de cuentas sin explicación,Soporte deficiente,No aplica,No aplica,"[contar él, tener]","[2021-2022, aplicación, banco, bloqueo, ciudad...",contar él
2,https://x.com/RevolutApp/status/19153142986219...,@TimChaney598055,NaN,WTF is happening with your app in Spain? I can...,2025-05-11T08:40:46.000Z,X,Revolut,@TimChaney598055,¿WTF está sucediendo con su aplicación en Espa...,0.994166,...,Unknown,0.648422,Low confidence,Problemas con el servicio de atención al clien...,Transferencias lentas,Transferencias rápidas,Soporte no responde,"[suceder, transferir]","[aplicación, cuenta, día, entrega, españa, fon...",suceder
3,https://x.com/RevolutApp/status/19153142986219...,@rach3kids,NaN,"Hello, I am trying to raise a complaint, but t...",2025-05-02T08:30:10.000Z,X,Revolut,@rach3kids,"Hola, estoy tratando de presentar una queja, p...",0.993253,...,Male,0.832703,High confidence,Problemas con el servicio de atención al clien...,Soporte deficiente,Confiable,Soporte no responde,"[colocar, presentar, tener, tratar]","[aplicación, correo, dirección, error, hola, q...",presentar
4,https://x.com/RevolutApp/status/19153142986219...,@GameofLife_ok,NaN,Tengo mi cuenta bloqueada desde hace días sin ...,2025-04-25T16:01:52.000Z,X,Revolut,@GameofLife_ok,Tengo mi cuenta bloqueada desde hace días sin ...,0.993852,...,Male,0.777338,High confidence,Bloqueo de cuentas sin explicación,Transferencias lentas,Transferencias rápidas,Tiempos de respuesta lentos,"[devolver, enviar, hacer, necesitar, pedir, te...","[ahorro, cuenta, dinero, día, explicación, fon...",tener
5,https://x.com/RevolutApp/status/19153142986219...,@BaroinEdouard,NaN,I use Revolut that is convenient for day to da...,2025-05-08T16:32:57.000Z,X,Revolut,@BaroinEdouard,Utilizo Revolut que es conveniente para el día...,0.994381,...,Male,0.995535,High confidence,Problemas con el servicio de atención al clien...,Soporte deficiente,Transferencias rápidas,Tiempos de respuesta lentos,"[hacer, indicar, perder, recibir, responder, t...","[chat, cosa, cuenta, desdén, día, fondo, pesad...",utilizar
6,https://x.com/RevolutApp/status/19153142986219...,@evrenuzayaldiz,NaN,Und Sie werden mein Revolut-Konto nicht genehm...,2025-04-24T13:48:22.000Z,X,Revolut,@evrenuzayaldiz,Y no aprobarás mi cuenta Revolut. Esto es muy ...,0.994093,...,Female,0.912035,High confidence,No aplica,No aplica,No aplica,Soporte no responde,[],"[aprobarás, cuenta, revolut]",None
7,https://x.com/RevolutApp/status/19153142986219...,@CriptosTW,NaN,Que tenéis que decir de la queja de un cliente...,2025-05-09T15:21:30.000Z,X,Revolut,@CriptosTW,Que tenéis que decir de la queja de un cliente...,0.993750,...,Unknown,0.673516,Low confidence,Bloqueo de cuentas sin explicación,Tarifas elevadas,Comisiones bajas,Atención al cliente pésima,"[correr, decir, querer, salir, tener]","[cliente, queja, vosotro]",correr
8,https://x.com/RevolutApp/status/19153142986219...,@Maximusprofitas,NaN,Are you insolvent? Have tried to transfer mone...,2025-05-02T08:43:35.000Z,X,Revolu

In [48]:
socialMediaData.groupby(['categories_client_attention']).size().reset_index(name='counts')

,categories_client_attention,counts
0,Atención al cliente pésima,618
1,No aplica,313
2,Publicidad engañosa,590
3,Soporte no responde,654
4,Tiempos de respuesta lentos,768


In [49]:
socialMediaData.to_csv(mainPath + '/socialMediaDataDivisas.csv', index=False, 
                       encoding='utf-8')


In [1]:
import os

In [2]:
import sys

In [7]:
sys.getwindowsversion()
os.name

'nt'

## Test

In [24]:
import pandas as pd
import numpy as np

In [25]:
mainPath = 'E:/Users/1167486/Local/proyectos/divisas/Scraper_divisas'


In [26]:
socialMediaData = pd.read_csv(mainPath + '/socialMediaDataDivisasbkp2.csv',
                       encoding='utf-8')


In [27]:
socialMediaData.head(100)

,link_id,username,name,comment,timestamp,social_media,app,username_to_gender,comment_es,confianza_idioma,...,gender,confidence,gender_liability,categories_time,categories_second_steps,categories_desired_categories,categories_client_attention,verbs,nouns,main_verb
0,https://x.com/RevolutApp/status/19153142986219...,@tawnyowlhillin1,NaN,BBC News - Revolut: 'I was careful and followe...,2025-04-24T08:04:43.000Z,X,Revolut,@tawnyowlhillin1,"BBC News - Revolut: ""Tuve cuidado y seguí las ...",0.959597,...,Unknown,0.550350,Low confidence,Problemas con la creación de cuentas y verific...,Soporte deficiente,Soporte rápido,Soporte no responde,"['perder', 'seguí', 'tener']","['bbc', 'criptografía', 'cuidado', 'instrucció...",tener
1,https://x.com/RevolutApp/status/19153142986219...,@denisPashchenko,NaN,With banking licence or like crypto payment sy...,2025-04-24T08:36:05.000Z,X,Revolut,@denisPashchenko,¿Con licencia bancaria o como sistema de pago ...,0.915091,...,Unknown,0.666464,Low confidence,Bloqueo de cuentas sin explicación,Soporte deficiente,No aplica,No aplica,"['contar él', 'tener']","['2021-2022', 'aplicación', 'banco', 'bloqueo'...",contar él
2,https://x.com/RevolutApp/status/19153142986219...,@TimChaney598055,NaN,WTF is happening with your app in Spain? I can...,2025-05-11T08:40:46.000Z,X,Revolut,@TimChaney598055,¿WTF está sucediendo con su aplicación en Espa...,0.994166,...,Unknown,0.648422,Low confidence,Problemas con el servicio de atención al clien...,Transferencias lentas,Transferencias rápidas,Soporte no responde,"['suceder', 'transferir']","['aplicación', 'cuenta', 'día', 'entrega', 'es...",suceder
3,https://x.com/RevolutApp/status/19153142986219...,@rach3kids,NaN,"Hello, I am trying to raise a complaint, but t...",2025-05-02T08:30:10.000Z,X,Revolut,@rach3kids,"Hola, estoy tratando de presentar una queja, p...",0.993253,...,Male,0.832703,High confidence,Problemas con el servicio de atención al clien...,Soporte deficiente,Confiable,Soporte no responde,"['colocar', 'presentar', 'tener', 'tratar']","['aplicación', 'correo', 'dirección', 'error',...",presentar
4,https://x.com/RevolutApp/status/19153142986219...,@GameofLife_ok,NaN,Tengo mi cuenta bloqueada desde hace días sin ...,2025-04-25T16:01:52.000Z,X,Revolut,@GameofLife_ok,Tengo mi cuenta bloqueada desde hace días sin ...,0.993852,...,Male,0.777338,High confidence,Bloqueo de cuentas sin explicación,Transferencias lentas,Transferencias rápidas,Tiempos de respuesta lentos,"['devolver', 'enviar', 'hacer', 'necesitar', '...","['ahorro', 'cuenta', 'dinero', 'día', 'explica...",tener
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://x.com/RevolutApp/status/19049405989366...,@Telcoin83,NaN,Please list #TEL so I can buy it digital bank,2025-03-29T11:34:58.000Z,X,Revolut,@Telcoin83,Por favor enumere #tel para que pueda comprarl...,0.991818,...,Male,0.944497,High confidence,Comisiones y cargos inesperados,Verificación compleja,App eficiente,Atención al cliente pésima,['comprar él'],"['#', 'banco', 'enumere', 'favor', 'tel']",comprar él
96,https://x.com/RevolutApp/status/19049405989366...,@leloupdesmonts,NaN,@revolut menteur !,2025-03-27T22:13:54.000Z,X,Revolut,@leloupdesmonts,@Revolut Menteur!,0.995324,...,Male,0.872827,High confidence,No aplica,No aplica,Tipo cambio bueno,No aplica,[],['menteur'],NaN
97,https://x.com/RevolutApp/status/19049405989366...,@onyi887,NaN,Your live in chat agents are the worst custome...,2025-04-03T22:40:27.000Z,X,Revolut,@onyi887,Sus agentes de chat en vivo son las peores per...,0.990808,...,Unknown,0.690872,Low confidence,Problemas con el servicio de atención al clien...,Soporte deficiente,Transferencias rápidas,Soporte no responde,"['necesitar', 'resolver', 'transferir']","['@revolutapp', 'agente', 'asunto', 'atención'...",transferir
98,https://x.com/RevolutApp/status/19049405989366...,@gryzonnn,NaN,who pays 150£ for bananas.,2025-03-27T15:02:16.000Z,X,Revolut,@gryzonnn,quien paga 150 £ por plátanos

In [28]:
socialMediaData['categories_time'] = np.where(socialMediaData['sentiment']=='Positivo', 'No aplica', socialMediaData['categories_time'])
socialMediaData['categories_second_steps'] = np.where(socialMediaData['sentiment']=='Positivo', 'No aplica', socialMediaData['categories_second_steps'])
socialMediaData['categories_client_attention'] = np.where(socialMediaData['sentiment']=='Positivo', 'No aplica', socialMediaData['categories_client_attention'])
socialMediaData['categories_desired_categories'] = np.where(socialMediaData['sentiment']=='Negativo', 'No aplica', socialMediaData['categories_desired_categories'])


socialMediaData['sentiment']= np.where(socialMediaData['emotion'].isin(['Alegría']), 'Positivo', socialMediaData['sentiment'])
socialMediaData['sentiment']= np.where(socialMediaData['emotion'].isin(['Enojo', 'Tristeza', 'Asco', 'Miedo']), 'Negativo', socialMediaData['sentiment'])
#socialMediaData['sentiment']= np.where(socialMediaData['emotion'].isin(['Neutral']), 'Neutral', socialMediaData['sentiment'])



In [29]:
socialMediaData.to_csv(mainPath + '/socialMediaDataDivisas.csv', index=False, 
                       encoding='utf-8')
